# Demo guide for package minplascalc

Welcome to minplascalc, a Python 3 package to assist with calculations of equilibrium compositions and thermophysical properties of thermal plasmas of the sort typically encountered in metallurgical processing applications.

## 1. Target audience

Plasma technology researchers and professionals with a basic knowledge of the Python programming language.

## 2. Description

Ionised gases or <i>plasmas</i> are used in many industrial applications such as arc welding, plasma spraying, and electric furnace metallurgy. Engineering plasmas typically operate at atmospheric pressures and temperatures of the order of 10<sup>4</sup> K. Thermal plasmas of the sort considered here are assumed to be in local thermodynamic equilibirum (LTE), meaning that a single unique temperature can be used to describe them (as opposed to two-temperature plasmas in which the electron temperature is distinct from the heavy-particle temperature). A major advantage of the LTE approximation is that all thermophysical properties of an equilibrium mixture of an arbitrary number of plasma species can be expressed as (complicated) functions of temperature and pressure only.

Knowledge of these thermophysical properties is of great value to engineers working in plasma technology. Such information is useful for simple design calculations, and is necessary input data for computational fluid dynamics and magnetohydrodynamics models of plasma systems. The calculation of composition and thence the thermophysical properties of a thermal plasma given some fundamental information about the species present is a well-understood but mathematically and numerically complex process - it is prone to error if performed manually, hence the need for this tool.

Things you <b>can</b> calculate with minplascalc:
* Statistical mechanics partition functions for individual species using information about the energy levels and excited states
* Equilibrium plasma composition in terms of number densities of a specified mixture of species, using the principle of Gibbs free energy minimisation at a specified temperature and pressure
* Density $\rho$, (relative) enthalpy h, and heat capacity C<sub>P</sub> at equilibrium conditions

Things you <b>can't</b> calculate with minplascalc:
* Compositions or thermophysical properties of two-temperature plasmas
* C<sub>P</sub> for arbitrary non-LTE compositions
* Plasmas of complex molecules or molecular ions consisting of more than two atoms
* Plasmas involving negatively-charged ions
* Transport or thermal radiation properties (see future versions though)

The package also includes some basic tools to format and import data for individual species obtained from the NIST Atomic Spectra and Chemistry WebBook databases, and store it for use by future simulations.

## 3. Worked examples

Some examples of how to use various parts of the minplascalc package follows.

*General utilities amd species calculations*
* <a href="#3.1a">Adding a new monatomic species data entry</a>
* <a href="#3.1b">Adding a new diatomic species data entry</a>
* <a href="#3.2">Calculate an internal partition function</a>

*Example 1 - oxygen plasma*
* <a href="#3.3">Calculate an equilibrium composition</a>
* <a href="#3.4">Calcluate density and heat capacity at equilibrium</a>

*Example 2 - silica & carbon monoxide plasma*
* <a href="#3.5">Calculate an equilibrium composition</a>
* <a href="#3.6">Calcluate density and heat capacity at equilibrium</a>

### 3.1 Adding a new species entry to your local data store

If you would like to do calculations using a plasma species that you haven't used before, you will need to generate a minplascalc data entry for it first. You only need to do this <i>once</i> - minplascalc will store the species data in a plain-text file formatted using JSON syntax, and it will then be available for use in any of your future calculations.

<a id="3.1a"></a>
#### 3.1a Add a monatomic species

minplascalc has built-in readers for handling data obtained from the energy levels section of NIST Atomic Spectra Database, which can be found at http://physics.nist.gov/PhysRefData/ASD/levels_form.html. The landing page looks like this:

<img src="demoimg/asd_landing_page.png" alt="NIST ASD landing page" style="width: 800px;"/>

You must then specify the atom or ion that you want to retrieve energy level information for. Let's get the data for the singly-charged oxygen cation species O<sup>+</sup>, which is "O II" in spectrographic terminology. Enter the identifier in the Spectrum field on the form:

<img src="demoimg/spectrum_identification.png" alt="Spectrum identification" style="width: 500px;"/>

Make sure you set Level Units to "cm-1", Format Output to "ASCII (text)", and uncheck everything in the output section except Level and J, like this:

<img src="demoimg/retrieval_settings.png" alt="Spectrum identification" style="width: 500px;"/>

Click the Retrieve Data button. You should see a page with the energy levels listed in a plain-text table. Click and drag to select everything from the first energy level line down to the last one before the first ionisation energy entry (these have no J value and are separated in boxes of ASCII dashes). While you're there, make a note of the ionisation energy value.

<table>
<td><img src="demoimg/level_data_1.png" alt="Spectrum identification" style="height: 400px;"/>
<td><img src="demoimg/level_data_2.png" alt="Spectrum identification" style="height: 400px;"/>
<td><img src="demoimg/level_data_3.png" alt="Spectrum identification" style="height: 400px;"/>
</table>

Copy and paste the selected content into a temporary text file. This has already been done for the oxygen cation, and the raw NIST data is located at minplascalc/data/species_raw/nist_O+. Running the following code snippet will add a data entry for the O<sup>+</sup> ion to minplascalc:

In [1]:
import minplascalc as mpc

mpc.buildMonatomicSpeciesJSON(
    name="O+",
    stoichiometry={"O": 1},
    molarMass=0.0159994,
    chargeNumber=1,
    ionisationEnergy=283270.9,
    nistDataFile="minplascalc/data/species_raw/nist_O+")


What's happening here? First we import the minplascalc package, then we call a function to build and save the species data entry for us. 

The function takes as arguments the species name (which is also used as the data entry's associated file name), a small dictionary describing the elemental stoichiometry of the species (in this case a single oxygen atom), the molar mass in kg/mol, the charge on the species in units of the fundamental charge (in this case 1 because O<sup>+</sup> is singly charged), the ionisation energy of the species in cm<sup>-1</sup>, and the path to the energy level text file we just generated from the NIST site. Internally to minplascalc the function reads the energy levels from the text file, assembles the species information into a Python dictionary format, and writes it out to a JSON file called "O+.json" on the path you're running this notebook from. Once you're done the JSON file should be moved to your local minplascalc data storage path.

After this process it will be possible to create an O<sup>+</sup> species object in any minplascalc calculation by importing it using either the explicit path to the JSON file, or (preferably) simply the name of the species provided the JSON file is stored in any of the standard minplascalc data paths - see later demos for examples.

<a id="3.1b"></a>
#### 3.1b Diatomic species

The procedure for generating a minplascalc data entry for a new diatomic species is more manual since the data required must be retrieved from several different locations in NIST. The majority comes from the NIST Chemistry WebBook, which is available at http://webbook.nist.gov/chemistry/form-ser/. The landing page looks like this:

[TODO]

<a id="3.2"></a>
### 3.2 Calculating partition functions for a single species

[TODO]

<a id="3.3"></a>
### 3.3 Calculating the composition of an equilibrium plasma

[TODO]

<a id="3.4"></a>
### 3.4 Calculating the density and heat capacity of an equilibrium plasma

[TODO]